In [ ]:
from google.colab import drive
drive.mount('/content/drive')

general_path = "/content/drive/My Drive/Colab Notebooks/"
%cd /content/drive/My\ Drive/Colab Notebooks/

## Reconstruction Error Function

In [ ]:
import os
import numpy as np
from PIL import Image

# Function that returns the Mean Reconstruction Error for an inpainting method, based on a list of ground truth images
def rec_err_per_id(gt_dir, inp_dir, lafin=False):
  gt_list = os.listdir(gt_dir) # list ground truth images

  avg_re = 0.0
  for subdir, dirs, files in os.walk(inp_dir):
    for dir in dirs:
      id_dir = os.path.join(inp_dir, dir) # get the id directory path
      
      inp_file = os.listdir(id_dir)[0]  # get the inpainted image in id_dir
      path = os.path.join(id_dir, inp_file)
            
      inp_im = Image.open(path) # open the inpainted image
      inp_im_arr = np.array(inp_im) / 255

      filename, ext = os.path.splitext(inp_file)

      if lafin:
        gt_filename = filename[:-4]
      else:
        gt_filename = filename[:-2]

      gt_file = gt_filename + ext
      gt_path = os.path.join(gt_dir, gt_file)      
      gt_im = Image.open(gt_path) # open the ground truth image
      gt_im_arr = np.array(gt_im) / 255

      im_pix_dif = gt_im_arr - inp_im_arr # calculate the pixel difference between the ground truth and the inpainted image

      im_re = 0.0
      # calculate the reconstruction error for each RGB channel
      for ch in [0,1,2]:
        pix_dif = im_pix_dif[:,:,ch] 
        abs_dif = np.linalg.norm(pix_dif, 'fro')
        sq_abs_dif = abs_dif **2
        
        gt_im_arr_ch = gt_im_arr[:,:,ch]
        abs_gt = np.linalg.norm(gt_im_arr_ch, 'fro')
        sq_abs_gt = abs_gt **2
        
        re = sq_abs_dif / sq_abs_gt # reconstruction error of each channel
        im_re += re  
      
      im_re = im_re / 3 # average reconstruction error of image
      avg_re += im_re

    avg_re = avg_re / len(gt_list) # average reconstruction error of method

    return round(avg_re, 3)

### PCPSFM

In [ ]:
gt_dir = os.path.join(general_path, "datasets/myceleba_ground_truth/")

res_dirs = ["results/myceleba_classic/", "results/myceleba_pcpf/", "results/myceleba_pcpfm/"]

for rdir in res_dirs: # traverse method directories
  # locate the directories containing only the inpainted images
  res_dir = os.path.join(general_path, rdir)
  L_dirs = [f for f in os.listdir(res_dir) if f.find("L") != -1]
  
  for dir in L_dirs:  # traverse each lamda subdirectory
    pathL = os.path.join(res_dir, dir)

    avg_re = rec_err_per_id(gt_dir, pathL)      
    print('Average Reconstruction Error ({}) = {}'.format(dir, avg_re))
  print()

### Lafin

In [ ]:
gt_dir = os.path.join(general_path, "datasets/myceleba_ground_truth/")

res_dir = os.path.join(general_path, "results/myceleba_lafin/")

# locate the directory containing only the inpainted images
path_inp = os.path.join(res_dir, "inp")
avg_re = rec_err_per_id(gt_dir, path_inp, True)      
print('Average Reconstruction Error ({}) = {}'.format(dir, avg_re))